In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
import models

In [4]:
E = 200 # GeV
M = 114 # GeV
c = 1

In [5]:
input_dimension = 50
data = np.random.beta(1, 2, (100_000, input_dimension))

In [6]:
data.mean(), data.std()

(0.3334279093694899, 0.2357638527313803)

In [7]:
data = np.loadtxt("data/events.txt")
# data = np.delete(data, [0, 4], axis=1)
input_dimension = data.shape[1]

In [8]:
data.shape

(100000, 10)

In [9]:
latent_dim = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vae = models.VAE(input_dimension, latent_dim, device=device).to(device)
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3) 


In [10]:
print(device, vae_optimizer.defaults)

cpu {'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}


In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(vae)

1375710

In [12]:
class PCDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        if self.transform:
            item = self.transform(item)
        return item

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test = train_test_split(data, test_size=0.33, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
print(X_train.mean(), X_train.std())
print(X_test.mean(), X_test.std())

-8.898010023653685e-16 0.8944271909999175
-7.742851612975211e-05 0.895897692447275


In [15]:
train_dataset = PCDataset(X_train)
val_dataset = PCDataset(X_test)

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

In [17]:
from tqdm.notebook import tqdm

In [18]:
vae.train()

for i in range(5):

    vae_loss_avg, recon_loss_avg, KL_loss_avg = 0, 0, 0

    for batch in tqdm(train_dataloader):

        vae_loss, recon_loss, KL_loss = vae.loss_function(batch.float().to(device))

        vae_loss_avg += vae_loss.detach().cpu().numpy() / len(train_dataloader)
        recon_loss_avg += recon_loss.detach().cpu().numpy() / len(train_dataloader)
        KL_loss_avg += KL_loss.detach().cpu().numpy() / len(train_dataloader)

        vae_optimizer.zero_grad()
        vae_loss.backward()
        vae_optimizer.step()

    print("Epoch:", i, f"Train Losses (avg.): {vae_loss_avg:.5f}, {recon_loss_avg:.5f}, {KL_loss_avg:.5f}")

  0%|          | 0/262 [00:00<?, ?it/s]

Epoch: 0 Train Losses (avg.): 0.02363, 0.02159, 0.00408


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch: 1 Train Losses (avg.): 0.00202, 0.00201, 0.00002


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch: 2 Train Losses (avg.): 0.00119, 0.00119, 0.00001


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch: 3 Train Losses (avg.): 0.00092, 0.00092, 0.00000


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch: 4 Train Losses (avg.): 0.00071, 0.00071, 0.00000


In [27]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9
count,67000.0,6.700000e+04,6.700000e+04,6.700000e+04,67000.0,6.700000e+04,6.700000e+04,6.700000e+04,6.700000e+04,6.700000e+04
mean,0.0,1.113537e-18,2.227074e-17,1.231678e-15,0.0,-1.113537e-18,-2.227074e-17,-1.231678e-15,1.684835e-15,-1.058348e-14
std,0.0,1.000007e+00,1.000007e+00,1.000007e+00,0.0,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00
min,0.0,-1.831403e+00,-1.822527e+00,-2.727541e+00,0.0,-1.831219e+00,-1.823497e+00,-1.223854e+00,-1.812103e+00,-1.737432e+00
25%,0.0,-8.308454e-01,-8.301694e-01,-5.746751e-01,0.0,-8.306023e-01,-8.327357e-01,-8.073244e-01,-7.482836e-01,-8.614282e-01
50%,0.0,-3.032000e-03,-9.418156e-04,2.668616e-01,0.0,3.032000e-03,9.418156e-04,-2.668616e-01,-1.514042e-01,2.592446e-03
75%,0.0,8.306023e-01,8.327357e-01,8.073244e-01,0.0,8.308454e-01,8.301694e-01,5.746751e-01,5.821515e-01,8.609991e-01
max,0.0,1.831219e+00,1.823497e+00,1.223854e+00,0.0,1.831403e+00,1.822527e+00,2.727541e+00,3.259955e+00,1.736025e+00


In [28]:
input = torch.tensor(X_test).float().to(device)

In [29]:
input

tensor([[ 0.0000,  0.4237,  0.6285,  ..., -1.0458, -1.1272, -0.6547],
        [ 0.0000,  1.2916, -1.2475,  ...,  0.3878,  0.4271, -0.8856],
        [ 0.0000,  0.5786, -0.7334,  ...,  2.4493,  2.4056, -0.7365],
        ...,
        [ 0.0000, -1.4568, -0.4385,  ..., -0.3469, -0.2293,  1.4110],
        [ 0.0000, -0.6135, -0.6833,  ...,  2.4596,  2.4223,  0.8053],
        [ 0.0000, -1.4379,  0.7935,  ..., -0.1192, -0.0132,  1.1767]])

In [30]:
output, _, _ = vae(input.float())

In [31]:
output.detach().cpu().numpy()

array([[ 1.3217241e-02,  4.4382074e-01,  6.5944922e-01, ...,
        -1.0627524e+00, -1.1222103e+00, -6.0649496e-01],
       [-2.1731444e-03,  1.2788570e+00, -1.2555132e+00, ...,
         3.7619704e-01,  4.1187513e-01, -8.8816988e-01],
       [ 6.6588148e-03,  5.5782509e-01, -7.0359039e-01, ...,
         2.4603777e+00,  2.4266779e+00, -7.7037632e-01],
       ...,
       [-5.2489787e-03, -1.4754137e+00, -4.7329682e-01, ...,
        -3.1574839e-01, -2.0386839e-01,  1.4131701e+00],
       [ 2.4863482e-03, -5.8077580e-01, -7.2928143e-01, ...,
         2.4626069e+00,  2.4513829e+00,  8.1002426e-01],
       [ 1.5574917e-03, -1.4049965e+00,  7.8634089e-01, ...,
        -1.3633825e-01, -2.0417370e-02,  1.1647389e+00]], dtype=float32)

In [32]:
import torch.nn.functional as F
F.mse_loss(input, output)

tensor(0.0006, grad_fn=<MseLossBackward0>)

In [33]:
scaler.inverse_transform(output.detach())

array([[100.01321724,  24.23540833,  36.14187745, ..., -91.84435419,
          0.43098729,   1.02291053],
       [ 99.99782686,  69.83096658, -68.89553001, ..., -19.0146941 ,
          1.37608325,   0.76815355],
       [100.00665881,  30.46039761, -38.62206902, ...,  86.47210612,
          2.61733228,   0.8746903 ],
       ...,
       [ 99.99475102, -80.56071906, -25.99025985, ..., -54.03618306,
          0.99674537,   2.84956856],
       [100.00248635, -31.71072746, -40.03124521, ...,  86.58493349,
          2.63255212,   2.30406168],
       [100.00155749, -76.71572432,  43.10199962, ..., -44.95568321,
          1.10976308,   2.6248784 ]])

In [34]:
scaler.inverse_transform(input.detach())

array([[100.        ,  23.13658617,  34.44653018, ..., -90.98425366,
          0.42789184,   0.97934015],
       [100.        ,  70.5283097 , -68.45617922, ..., -18.42577445,
          1.38547971,   0.77049017],
       [100.        ,  31.59496339, -40.25694932, ...,  85.91354427,
          2.60437412,   0.90537254],
       ...,
       [100.        , -79.54435376, -24.08205382, ..., -55.61250058,
          0.98108033,   2.84761481],
       [100.        , -33.4996952 , -37.5090224 , ...,  86.43404649,
          2.61463363,   2.2997919 ],
       [100.        , -78.5114486 ,  43.49693276, ..., -44.09046208,
          1.11419003,   2.6356681 ]])